## Offline dictionary attack on simplified SRP
```
S
x = SHA256(salt|password)
    v = g**x % n

C->S
I, A = g**a % n

S->C
salt, B = g**b % n, u = 128 bit random number

C
x = SHA256(salt|password)
    S = B**(a + ux) % n
    K = SHA256(S)

S
S = (A * v ** u)**b % n
    K = SHA256(S)

C->S
Send HMAC-SHA256(K, salt)
S->C
Send "OK" if HMAC-SHA256(K, salt) validates
```

Note that in this protocol, the server's "B" parameter doesn't depend on the password (it's just a Diffie Hellman public key).

Make sure the protocol works given a valid password.

Now, run the protocol as a MITM attacker: pose as the server and use arbitrary values for b, B, u, and salt.

Crack the password from A's HMAC-SHA256(K, salt).


## Parameter negotiation
Carol and Steve agree on $N$ and $g$.

In [1]:
# A large safe prime (N = 2q+1, where q is prime)
# All arithmetic is done modulo N
# (generated using "openssl dhparam -text 1024")
N = """00:c0:37:c3:75:88:b4:32:98:87:e6:1c:2d:a3:32:
       4b:1b:a4:b8:1a:63:f9:74:8f:ed:2d:8a:41:0c:2f:
       c2:1b:12:32:f0:d3:bf:a0:24:27:6c:fd:88:44:81:
       97:aa:e4:86:a6:3b:fc:a7:b8:bf:77:54:df:b3:27:
       c7:20:1f:6f:d1:7f:d7:fd:74:15:8b:d3:1c:e7:72:
       c9:f5:f8:ab:58:45:48:a9:9a:75:9b:5a:2c:05:32:
       16:2b:7b:62:18:e8:f1:42:bc:e2:c3:0d:77:84:68:
       9a:48:3e:09:5e:70:16:18:43:79:13:a8:c3:9c:3d:
       d0:d4:ca:3c:50:0b:88:5f:e3"""

N = int("".join(N.split()).replace(":", ""), 16)
g = 2 # A generator modulo N

In [2]:
N.bit_length()

1024

In [3]:
I = b'carol'
password = b'foobar'

## Client registration
Carol sends salt and verifier $v$ to Steve.

Steve stores salt and $v$.

In [4]:
import os
import hashlib

In [5]:
salt = os.urandom(8)
salt.hex()

'a0ce6b3558fac047'

In [6]:
int.from_bytes(salt, byteorder='big').bit_length() # Confirm 64-bit salt. :-\

64

In [7]:
sha256 = hashlib.sha256()
sha256.update(salt)
sha256.update(password)
x = sha256.digest()
x.hex()

'1d1ad2c18f08d35ab45ac6a99f888ed5e38eb11170bd35b9ec155989353c73b8'

In [8]:
x = int.from_bytes(x, byteorder="big")

In [9]:
v = pow(g, x, N)
v

93207369554840988258178257633432713421800657908911100487525262719273024365871723892842350982738538542834877188638923251097689279124732123677680707089412897744314411883369981779489358956443468394620931103270066650563690350039158551434149252138468494883550764145678096967702669043221727010142023128468944792628

## Proof of Password

Carol sends $I$ and $A = g^a\mod{N}$.

In [10]:
from secrets import randbelow

In [11]:
a = randbelow(N)
A = pow(g, a, N)

Steve sends salt, $B = g^b\mod{N}$, and $u$ (a 128-bit random number).

In [12]:
b = randbelow(N)
B = pow(g, b, N)

In [13]:
u = int.from_bytes(os.urandom(16), byteorder='big')
u

222334989561627904865494199452972795048

In [14]:
from Crypto.Util.number import long_to_bytes

Carol calculates $S = B^{a + ux}\mod{N}$.

In [15]:
S_c = pow(B, a + u * x, N)
K_c = hashlib.sha256(long_to_bytes(S_c)).digest()
K_c.hex()

'11aa789639e341a33e9994a10780c995c0f89825d27fe7da73a81897d638e2e6'

Steve calculates $S = (A + v^u)^b\mod{N}$.

In [16]:
S_s = pow(A * pow(v, u, N), b, N)
K_s = hashlib.sha256(long_to_bytes(S_s)).digest()
K_s.hex()

'11aa789639e341a33e9994a10780c995c0f89825d27fe7da73a81897d638e2e6'

In [17]:
import hmac

Carol sends $MAC(K_c, salt)$.

In [18]:
mac = hmac.digest(K_c, salt, digest='sha256')

Steve verifies against $MAC(K_s, salt)$.

In [19]:
hmac.compare_digest(mac, hmac.digest(K_s, salt, digest='sha256'))

True